In [1]:
import pandas as pd
import json,string

## Initial Twitter Miner

In [2]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [3]:
ckey = "ZIFdnT2XHHgHTzuQvAs0kqiW8"
csecret = "rmB7Bus6kBV31a4xrXCTbWVVuMPdSMvRcCJcb7TKemY9xPVz4F"
atoken = "101726O65-5y1ByFalKXxBUEI9rClMNwKD42OLig30n7utIk5I"
asecret = "PTfnWxzyXFnt13I9tp53KEDGjosljBq63wyecZn933S5h"

In [4]:
auth = tweepy.OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
api.rate_limit_status()['resources']['application']

{'/application/rate_limit_status': {'limit': 180,
  'remaining': 178,
  'reset': 1506011636}}

In [6]:
def stream_processing(tweet):
    global db
    tweet = json.loads(tweet)
    
    if tweet['lang']=='en':
        try:
            tweet = tweet['retweeted_status']
            if tweet['id'] in db['id']:
                # for instance, increase the counter of retweeted
                # could be assessed like in previous case or could be used another way
                pass
            else:
                try:
                    # could be added some extra fields like "retweeted" 
                    db = db.append([[tweet['id'],str(tweet['created_at']),tweet['text'],\
                                     tweet['user']['followers_count'],tweet['user']['description'],\
                                    # etc.
                                    ]],ignore_index=True)
                    
                    # another way exists: 
                    # FILE = open('database.txt','w')
                    # FILE.write(tweet)
                except:                
                    print('_1 SMTH WRONG!')
        except:
            try:
                db = db.append([[tweet['id'],str(tweet['created_at']),tweet['text'],\
                                tweet['user']['followers_count'],tweet['user']['description'],\
                                # etc.
                                ]],ignore_index=True)
            except:                
                print('_2 SMTH WRONG!')

In [7]:
class StdOutListener(StreamListener):
    def on_data(self, data):        
        stream_processing(data)
        return True

    def on_error(self, status):
        print(status)

In [11]:
db = pd.DataFrame()

In [12]:
listener = StdOutListener()
stream = Stream(auth, listener)
stream.filter(track=['bitcoin','ethereum','blockchain','EOS','Monero','btc','ico','ripple','dash','IOTA','Stratis'])

KeyboardInterrupt: 

In [13]:
db.columns = ['tweet_id','tweet_date_time','tweet_text','user_followers_count','user_description']
db.head()

,tweet_id,tweet_date_time,tweet_text,user_followers_count,user_description
0,910904635438837760,Thu Sep 21 16:32:43 +0000 2017,Walbrock Research: There are only 21m Bitcoins...,235,Win Free Bitcoins every hour! https://goo.gl/w...
1,910886094862012416,Thu Sep 21 15:19:03 +0000 2017,Big announcements &amp; preview coming from #S...,2738,The Substratum Network will bring the decentra...
2,910893080097603584,Thu Sep 21 15:46:48 +0000 2017,How to Add Bitindia Tokens using MyEtherWallet...,16554,None
3,910904637435326464,Thu Sep 21 16:32:44 +0000 2017,"TopFiveTraders: RT StakepoolCom ""More bad news...",235,Win Free Bitcoins every hour! https://goo.gl/w...
4,903996075979264001,Sat Sep 02 15:00:34 +0000 2017,bitJob ICO launches Sept 12th. Learn more at ...,5432,None


In [ ]:
db.to_csv('database.csv')

## Initial CoinDesk Miner

In [14]:
from bs4 import BeautifulSoup
import urllib.request
import re,feedparser

In [15]:
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer

In [16]:
def preprocess_coin_desk(query):
    query = re.sub('<.*?>|','',str(query))
    query = re.sub('\n',' ',query)
    query = re.sub('!function.*',' ',query)
    query = query.encode('ascii', 'ignore').decode('utf-8')    
    query = re.sub('\\\'|\"|\'','',query)    
    query = re.sub('[?.!-$£#,/();:]|[0-9]+',' ',query)
    query = query.strip().lower().split()
    
    # for example way 
    st = LancasterStemmer()    
    query = [st.stem(word) for word in query if word not in stopwords.words('english') and len(word)>1]
    
    return query

In [17]:
rss_href = 'https://feeds.feedburner.com/CoinDesk'

In [18]:
feed = feedparser.parse(rss_href)

In [19]:
db = pd.DataFrame()

In [20]:
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/6.0"

opener = AppURLopener()    
    
for entries in feed['entries']:
    try:
        response = opener.open(entries['link'])
        soup = BeautifulSoup(response.read(),"html.parser")
        db = db.append([[re.findall('p=.*',entries['id'])[0][2:],entries['published'],\
                         preprocess_coin_desk(soup.findAll('div', { 'class' : 'article-content-container noskimwords' })[0]),\
                        [tags['term'].lower() for tags in entries['tags']],\
                        entries['author'].lower()]],\
                       ignore_index=True)
    except:
            print('_3 SMTH WRONG!')

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:4: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


In [21]:
db.columns = ['post_id','post_datetime','body_text_preproc','tags','author']
db.head()

,post_id,post_datetime,body_text_preproc,tags,author
0,287445,"Thu, 21 Sep 2017 16:15:38 +0000","[combin, silicon, valley-based, startup, accel...","[use cases & verticals, news, business news, s...",rachel rose o'leary
1,287386,"Thu, 21 Sep 2017 15:00:15 +0000","[germany, cent, bank, publ, new, research, pap...","[news, central banking, distributed ledger tec...",stan higgins
2,287435,"Thu, 21 Sep 2017 14:00:22 +0000","[rebound, bitcoin-u, doll, btc, usd, exchang, ...","[features, markets, bitcoin, markets news, pri...",omkar godbole
3,287423,"Thu, 21 Sep 2017 13:00:15 +0000","[urbit, galact, inspir, network, cloud, serv, ...","[news, ethereum, technology news, smart contra...",rachel rose o'leary
4,287193,"Thu, 21 Sep 2017 12:00:07 +0000","[vent, arm, on, recogn, nam, tech, industry, h...","[features, venture capital, investments, marke...",michael del castillo


In [ ]:
### Unfortunately, not working 

# response = opener.open('https://www.coindesk.com/proof-of-space-bittorrent-creator-publishes-eco-friendly-mining-paper/')
# soup = BeautifulSoup(response.read(),"html.parser")    
# a = soup.findAll('ul', { 'class' : 'share-bar' })[0]
# share_title_list = {}
# for li in a.findAll('li')[:-2]:
#     print(re.compile("<li class=(.*)>").search(str(li)).group(1))
#     print(re.compile("<span class=\"count\">(.*)</span>").search(str(li)).group(1))

# [
#     "twitter"
#     0
#     "facebook"
#     0
#     "google-plus"
#     0
#     "linkedin"
#     0
#     "reddit"
#     12
# ]